In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Import necessary modules
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv
from IPython.core.display import HTML
from IPython.display import display

# Import the transcription service modules
from tnh_scholar.audio_processing.transcription_service import (
    DiarizationChunker,
    TranscriptionFormatConverter,
    TranscriptionServiceFactory,
)

# Load environment variables from .env file
load_dotenv()


True

In [3]:
import logging

from tnh_scholar.logging_config import setup_logging

logger = setup_logging(log_level=logging.DEBUG)

In [4]:
# Replace with your audio file path
working_dir = Path.home() / "Desktop" / "transcription_wouter"
audio_file = working_dir / "qa_sr_abbess_wh_sh.mp3"
if not audio_file.exists:
    raise FileNotFoundError("Not found.")

In [5]:
def run_transcription(audio_file_path, provider="whisper", options=None):
    """
    Transcription with the specified provider.
    
    Args:
        audio_file_path: Path to the audio file
        provider: 'whisper' or 'assemblyai'
    
    Returns:
        Transcription result
    """
    if options is None:
        options = {}
    # Create the transcription service
    service = TranscriptionServiceFactory.create_service(provider=provider)

    # Print some info
    print(f"Testing transcription with {provider} service...")
    print(f"Audio file: {audio_file_path}")

    return service.transcribe(audio_file_path, options)

In [6]:
def gen_srt(audio_file_obj, provider="whisper", language=None, local_convert=False):
    """
    generate srt
    """
    format_type = "srt"
    # Create the transcription service
    service = TranscriptionServiceFactory.create_service(provider=provider)

    # Print some info
    print(f"Running {format_type.upper()} generation with {provider} service...")
    print(f"Audio file: {audio_file_obj}")

    transcription_options = {"language": language} if language else None
    
    # Generate the formatted transcription
    # use the local format converter if specified
    if local_convert:
        converter = TranscriptionFormatConverter()
        transcript = service.transcribe(audio_file_obj, options=transcription_options)
        return converter.convert(transcript)
        
    return service.transcribe_to_format(
        audio_file_obj, 
        format_type=format_type,
        transcription_options=transcription_options
    )
    

In [7]:
# Test Whisper transcription
options = {
    "language": "vi",
    "timestamp_granularities": ["word"] 
           }
whisper_result = run_transcription(audio_file, provider="whisper", options=options)



2025-04-26 21:36:54,610 - tnh.tnh_scholar.audio_processing.transcription_service.assemblyai_service - DEBUG - Created logger with name: tnh.tnh_scholar.audio_processing.transcription_service.assemblyai_service
2025-04-26 21:36:54,785 - tnh.tnh_scholar.audio_processing.transcription_service.whisper_service - DEBUG - Created logger with name: tnh.tnh_scholar.audio_processing.transcription_service.whisper_service
2025-04-26 21:36:54,787 - tnh.tnh_scholar.audio_processing.transcription_service.whisper_service - DEBUG - API key updated
2025-04-26 21:36:54,787 - tnh.tnh_scholar.audio_processing.transcription_service.whisper_service - INFO - Transcribing audio with Whisper API using model: whisper-1


Testing transcription with whisper service...
Audio file: /Users/phapman/Desktop/transcription_wouter/qa_sr_abbess_wh_sh.mp3


2025-04-26 21:37:39,253 - tnh.tnh_scholar.audio_processing.transcription_service.whisper_service - DEBUG - model_dumping...
2025-04-26 21:37:39,257 - tnh.tnh_scholar.audio_processing.transcription_service.whisper_service - DEBUG - Dumped.
2025-04-26 21:37:39,258 - tnh.tnh_scholar.audio_processing.transcription_service.whisper_service - DEBUG - converting duration...
2025-04-26 21:37:39,259 - tnh.tnh_scholar.audio_processing.transcription_service.whisper_service - DEBUG - converted.
2025-04-26 21:37:39,260 - tnh.tnh_scholar.audio_processing.transcription_service.whisper_service - INFO - Transcription completed successfully


In [28]:
[utter['text'] for  utter in whisper_result['utterances']]

[]

In [8]:
list(whisper_result["words"])

[{'word': 'Dạ', 'start_ms': 0, 'end_ms': 500},
 {'word': 'con', 'start_ms': 699, 'end_ms': 699},
 {'word': 'kính', 'start_ms': 699, 'end_ms': 920},
 {'word': 'Bạch', 'start_ms': 920, 'end_ms': 1159},
 {'word': 'sư', 'start_ms': 1159, 'end_ms': 1279},
 {'word': 'ông', 'start_ms': 1279, 'end_ms': 1539},
 {'word': 'con', 'start_ms': 1779, 'end_ms': 1919},
 {'word': 'kính', 'start_ms': 1919, 'end_ms': 2079},
 {'word': 'thưa', 'start_ms': 2079, 'end_ms': 2319},
 {'word': 'quý', 'start_ms': 2319, 'end_ms': 2440},
 {'word': 'thầy', 'start_ms': 2440, 'end_ms': 2700},
 {'word': 'quý', 'start_ms': 3000, 'end_ms': 3059},
 {'word': 'sư', 'start_ms': 3059, 'end_ms': 3200},
 {'word': 'cô', 'start_ms': 3200, 'end_ms': 3480},
 {'word': 'và', 'start_ms': 3480, 'end_ms': 4320},
 {'word': 'toàn', 'start_ms': 4320, 'end_ms': 4480},
 {'word': 'thế', 'start_ms': 4480, 'end_ms': 4599},
 {'word': 'đại', 'start_ms': 4599, 'end_ms': 4820},
 {'word': 'chúng', 'start_ms': 4820, 'end_ms': 5159},
 {'word': 'thưa', 

In [9]:
# Examine specific parts of the result
print("\nExamining detailed result:")
print(f"Language: {whisper_result['language']}")
print(f"Audio duration: {whisper_result.get('audio_duration_ms', 'Not available')}")
print(f"Word count: {len(whisper_result.get('words', []))}")


Examining detailed result:
Language: vietnamese
Audio duration: 381489
Word count: 821


In [10]:
# Test 'hand' format conversion to SRT
converter = TranscriptionFormatConverter()
srt_conv = converter.convert(whisper_result)

2025-04-26 21:38:14,851 - tnh.tnh_scholar.audio_processing.transcription_service.format_converter - WARNING - Found an empty segment. Unexpected value.


In [11]:
print(srt_conv)

2025-04-26 21:46:25,284 - tnh.tnh_scholar.audio_processing.transcription_service.format_converter - DEBUG - Created logger with name: tnh.tnh_scholar.audio_processing.transcription_service.format_converter


1
00:00:00,000 --> 00:00:03,480
Dạ con kính Bạch sư ông con kính thưa quý thầy quý sư cô

2
00:00:03,480 --> 00:00:08,500
và toàn thế đại chúng thưa sư cô chủ trì chúng con

3
00:00:08,500 --> 00:00:13,279
rất là biết ơn hôm nay sư cô đã cho chúng con một bối chia

4
00:00:13,279 --> 00:00:15,760
sẻ về cuộc đời của sư cô

5
00:00:17,840 --> 00:00:22,840
và chúng ta đang ở trong một tuần lễ của Khoá

6
00:00:22,840 --> 00:00:27,280
học nấu ăn trong chánh niệm do Tư viện Lọc Uyển tổ chức

7
00:00:27,280 --> 00:00:34,020
và hôm nay là ngày 27 tháng 2 năm 2025

8
00:00:34,319 --> 00:00:36,939
chúng ta đang ở trong thiền đường Thái Bình Dương của

9
00:00:36,939 --> 00:00:41,959
Tư viện Lọc Uyển và hôm nay bố chia sẻ sẽ được gửi đến

10
00:00:41,959 --> 00:00:45,380
quý thầy quý sư cô cũng như các bạn thiền sinh trong Khoá

11
00:00:45,380 --> 00:00:49,479
học nấu ăn chánh niệm đây là sư cô chủ trì Trung Chính

12
00:00:49,939 --> 00:00:53,380
sư cô chủ trì của Sống Trong Sáng Tư viện Lọc U

In [6]:
# Test SRT generation with Whisper
whisper_srt = gen_format(audio_file, provider="whisper", format_type="srt")


Testing SRT generation with whisper service...
Audio file: /Users/phapman/Desktop/wouter_video/dt_trim_enh_short.wav


In [8]:
print(whisper_srt)

1
00:00:00,000 --> 00:00:14,460
Thưa đại chúng, vừa rồi Tù Viện có mấy ngày trong khóa tu nấu ăn online

2
00:00:14,460 --> 00:00:20,180
Mục đích là để giới thiệu cho cộng đồng những người ăn chay

3
00:00:20,180 --> 00:00:27,700
những món ăn được nấu ở trong Tù Viện

4
00:00:27,700 --> 00:00:38,779
Thưa đại chúng, Thành nghị Phố Hội chia sẻ ngày hôm nay về cước ăn và đi chợ

5
00:00:38,779 --> 00:00:45,419
Đó cũng có một chút đặc biệt là bởi vì chúng ta là một cộng đồng ăn chay

6
00:00:45,419 --> 00:00:51,980
Chay ở đây có bao gồm veggie và bacon

7
00:00:51,980 --> 00:01:04,139
Và mọi người cũng rất là nhiều người có hứng thú xem là các thầy, các sư cô họ ăn cái gì hằng ngày ở trong Tù Viện

8
00:01:04,139 --> 00:01:11,059
Và cũng cả họ đi chợ như thế nào nữa

9
00:01:12,059 --> 00:01:23,059
Có lẽ là bởi vì Pháp Hội là người có thâm niên đi chợ lâu nhất so với đại chúng

10
00:01:23,059 --> 00:01:29,059
Cho nên xin cảm ơn đại chúng những kinh tượng để chia sẻ cho lĩnh vực này

11
00

In [12]:
from tnh_scholar.utils.file_utils import write_str_to_file

out_srt = working_dir / "Dharma Talk Br. Phap Hoi (for transcription) 2-bit.srt"
write_str_to_file(out_srt, whisper_srt)